In [3]:
import sys, os
import requests
import json
import numpy as np
import time
import pandas as pd
import datetime as dt
from config import api_key_inCites
from config import api_key_WOS
import clarivate.wos_journals.client
from clarivate.wos_journals.client.api import journals_api
from clarivate.wos_journals.client.model.journal_list import JournalList
from pprint import pprint

## Pip install the Clarivate Journal Client
## pip install git+https://github.com/clarivate/wosjournals-python-client.git

In [4]:
## Print files in current dirctory
for i in os.listdir('./'):
    print(i)

.git
.gitignore
.ipynb_checkpoints
CMMS_ILL_Requests_Turnaways.ipynb
config.py
data
LICENSE
output
README.md
__pycache__


In [5]:
## Use this to SQL query in Elements Reporting Database for faculty publication data
## https://github.com/kglibrarian/symplecticelementssql/blob/master/Data%20Source%20Publication%20IDs%20Report%20by%20Group%20of%20Authors.md

## Upload the resulting .csv from Elements

elements_data_path = "data/2024_05-16_FSM_Pubs_2019_to_2023.csv"
##elements_data_path = "data/2023_01-12_FSM_Pubs_2018_to_2022.csv"

## Read the CSV file and store into Pandas DataFrame 
elements_data_df = pd.read_csv(elements_data_path, encoding = "ISO-8859-1")
## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
elements_data_df.columns =  elements_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
elements_data_df.head()

C:\Users\keg827\AppData\Local\Temp\ipykernel_10268\1519900468.py:10: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  elements_data_df = pd.read_csv(elements_data_path, encoding = "ISO-8859-1")
C:\Users\keg827\AppData\Local\Temp\ipykernel_10268\1519900468.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  elements_data_df.columns =  elements_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
C:\Users\keg827\AppData\Local\Temp\ipykernel_10268\1519900468.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  elements_data_df.columns =  elements_data_df.c

,group_id,user_id,name,position,department,school,employee_id,net_id,is_current_staff,is_academic,...,doi,issn,eissn,types,type,scopus,web_of_science,crossref,pubmed,europe_pubmed_central
0,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,10.1080/01639374.2019.1699880,0163-9374,1544-4554,Journal Article,Journal article,2-s2.0-85076392535,NaN,10.1080/01639374.2019.1699880,NaN,NaN
1,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,10.1080/0361526X.2019.1583045,0361-526X,1541-1095,Journal Article,Journal article,2-s2.0-85065402087,NaN,10.1080/0361526x.2019.1583045,NaN,NaN
2,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,10.1080/08989621.2020.1779591,0898-9621,1545-5815,"Article, Journal",Journal article,2-s2.0-85087477966,WOS:000549691200001,10.1080/08989621.2020.1779591,32602379.0,MED:32602379
3,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,10.1074/jbc.W119.009587,NaN,1083-351X,Erratum,Journal article,2-s2.0-85068977968,NaN,10.1074/jbc.w119.009587,31253689.0,MED:31253689
4,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,10.1126/scitranslmed.abe4282,1946-6234,1946-6242,"Article, Journal",Journal article,2-s2.0-85097911743,WOS:000599969700007,10.1126/scitranslmed.abe4282,33257409.0,MED:33257409


In [6]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(elements_data_df.index))

## Check how many nans are in each column
elements_data_df.isnull().sum(axis = 0)

Number of rows in the dataframe:  69906


group_id                     0
user_id                      0
name                         0
position                     0
department                 265
school                      67
employee_id                  0
net_id                       0
is_current_staff             0
is_academic                  0
publication_id               0
publication-date             0
doi                       4395
issn                     10881
eissn                    10385
types                      837
type                         0
scopus                    6760
web_of_science           21445
crossref                  4239
pubmed                   10107
europe_pubmed_central     9933
dtype: int64

In [9]:
## Make a column based on year
## 1. Change publication_date column from object to a datetime format
elements_data_df['publication-date'] = pd.to_datetime(elements_data_df['publication-date'], format='%m/%d/%Y')
##elements_data_df['publication_date'] = pd.to_datetime(elements_data_df['publication_date'], format='%m/%d/%Y')

## 2. Extract year and create new column from publication_date
elements_data_df['year'] = elements_data_df['publication-date'].dt.year 
##elements_data_df['year'] = elements_data_df['publication_date'].dt.year 
elements_data_df.head()

,group_id,user_id,name,position,department,school,employee_id,net_id,is_current_staff,is_academic,...,issn,eissn,types,type,scopus,web_of_science,crossref,pubmed,europe_pubmed_central,year
0,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,0163-9374,1544-4554,Journal Article,Journal article,2-s2.0-85076392535,NaN,10.1080/01639374.2019.1699880,NaN,NaN,2020
1,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,0361-526X,1541-1095,Journal Article,Journal article,2-s2.0-85065402087,NaN,10.1080/0361526x.2019.1583045,NaN,NaN,2019
2,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,0898-9621,1545-5815,"Article, Journal",Journal article,2-s2.0-85087477966,WOS:000549691200001,10.1080/08989621.2020.1779591,32602379.0,MED:32602379,2021
3,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,NaN,1083-351X,Erratum,Journal article,2-s2.0-85068977968,NaN,10.1074/jbc.w119.009587,31253689.0,MED:31253689,2019
4,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,1946-6234,1946-6242,"Article, Journal",Journal article,2-s2.0-85097911743,WOS:000599969700007,10.1126/scitranslmed.abe4282,33257409.0,MED:33257409,2020


In [10]:
## Look up data based on a particular EISSN
elements_data_df.loc[elements_data_df['issn'] == '1545-9616']

,group_id,user_id,name,position,department,school,employee_id,net_id,is_current_staff,is_academic,...,issn,eissn,types,type,scopus,web_of_science,crossref,pubmed,europe_pubmed_central,year
2334,15,727,Amy Paller,Professor,Dermatology;,Feinberg School of Medicine,1006032,apaller,1,1,...,1545-9616,NaN,"Article, Journal",Journal article,2-s2.0-85062076026,WOS:000458712900001,NaN,30794360.0,MED:30794360,2019
2543,15,727,Amy Paller,Professor,Dermatology;,Feinberg School of Medicine,1006032,apaller,1,1,...,1545-9616,NaN,Journal Article,Journal article,2-s2.0-85160985664,NaN,10.36849/jdd.7108,37276163.0,MED:37276163,2023
3631,15,1036,Joan Guitart,Professor,Dermatology;,Feinberg School of Medicine,1008991,jgu906,1,1,...,1545-9616,NaN,Letter,Journal article,2-s2.0-85102689438,NaN,10.36849/jdd.5756,33683074.0,MED:33683074,2021
4928,15,1577,Roopal Kundu,Professor,Dermatology;,Feinberg School of Medicine,1014634,rav502,1,1,...,1545-9616,NaN,"Article, Journal",Journal article,2-s2.0-85099007951,WOS:000606610600004,10.36849/jdd.2020.5488,33346515.0,MED:33346515,2020
4933,15,1577,Roopal Kundu,Professor,Dermatology;,Feinberg School of Medicine,1014634,rav502,1,1,...,1545-9616,NaN,"Article, Journal",Journal article,2-s2.0-85110957426,WOS:000677732600013,NaN,34232004.0,MED:34232004,2021
4937,15,1577,Roopal Kundu,Professor,Dermatology;,Feinberg School of Medicine,1014634,rav502,1,1,...,1545-9616,NaN,"Editorial Material, Editorial material, Journal",Journal article,2-s2.0-85133775382,WOS:000827133500016,NaN,35816061.0,MED:35816061,2022
5525,15,1854,Jerome Garden,"Professor, Clinical",Dermatology;,Feinberg School of Medicine,1017474,jmg340,1,1,...,1545-9616,NaN,Journal Article,Journal article,2-s2.0-85085904859,NaN,10.36849/jdd.2020.4741,32484632.0,MED:32484632,2020
5649,15,1911,Timothy Kuzel,"Professor, Clinical","Medicine, Hematology Oncology Division;",Feinberg School of Medicine,1018134,Timothy.Kuzel.1018134@fake.galte,1,1,...,1545-9616,NaN,Letter,Journal article,2-s2.0-85102689438,NaN,10.36849/jdd.5756,33683074.0,MED:33683074,2021
6508,15,2330,Anthony Mancini,Professor,Pediatrics;,Feinberg School of Medicine,1022881,ajm898,1,1,...,1545-9616,NaN,"Article, Journal",Journal article,2-s2.0-85121960715,WOS:000721535300005,10.36849/jdd.6383,34784125.0,MED:34784125,2021
10370,15,3072,Amy Taub,"Assistant Professor, Clinical",Dermatology;,Feinberg School of Medicine,1036595,aft180,1,1,...,1545-9616,NaN,Journal Article,Journal article,2-s2.0-85078570531,NaN,10.36849/jdd.2020.4568,31985195.0,MED:31985195,2020


In [11]:
## Summarize how many faculty have published in a journal by year
## Make a new dataframe with subset of data, including user_id, publication_id, issn, eissn, and year
## Our backbone ID is the eissn, and then also year
## a) If publication ID is listed more than once for each journal in each year, we only want to count it once. 
## b) If user ID is listed more than once for each journal in each year, we only want to count it once.

unique_EISSN_publications_df = elements_data_df.groupby(['year', 'eissn']).agg({'publication_id': 'nunique', 
                                                                          'user_id':'nunique' }).rename(columns={'publication_id': 'Count of Unique Publications',
                                                                                                                'user_id': 'Count of Unique Authors'})
unique_EISSN_publications_df.head(100)


Count of Unique Publications  Count of Unique Authors
year eissn                                                           
2019 0148-396X                             1                        1
     0807-7096                             1                        1
     0893-7400                             1                        1
     0925-4005                             1                        1
     0942-0940                             2                        1
...                                      ...                      ...
     1098-8947                             2                        2
     1098-8963                             3                        9
     1098-8971                             2                        2
     1098-898X                             3                        4
     1098-9021                             2                        2

[100 rows x 2 columns]

In [12]:
## Reset the index so year and eissn are columns
unique_EISSN_publications_df.reset_index(level=['year','eissn'], inplace=True)
unique_EISSN_publications_df.head(100)



,year,eissn,Count of Unique Publications,Count of Unique Authors
0,2019,0148-396X,1,1
1,2019,0807-7096,1,1
2,2019,0893-7400,1,1
3,2019,0925-4005,1,1
4,2019,0942-0940,2,1
...,...,...,...,...
95,2019,1098-8947,2,2
96,2019,1098-8963,3,9
97,2019,1098-8971,2,2
98,2019,1098-898X,3,4


In [14]:
## Summarize across 5 years how many unique faculty have published in a journal

unique_EISSN_publications_all_df = elements_data_df.groupby(['eissn']).agg({'publication_id': 'nunique', 
                                                                          'user_id':'nunique' }).rename(columns={'publication_id': 'Count of Unique Publications 2019-2023',
                                                                                                                'user_id': 'Count of Unique Authors 2019-2023'})
unique_EISSN_publications_all_df.head(100)

,Count of Unique Publications 2019-2023,Count of Unique Authors 2019-2023
eissn,,
0148-396X,1,1
0219-1032,1,1
0807-7096,1,1
0893-7400,4,5
0925-4005,1,1
...,...,...
1097-0177,2,2
1097-0193,22,27
1097-0215,19,26


In [15]:
## Pivot so that the data is provided by Year

pivot_unique_EISSN_publications_df = unique_EISSN_publications_df.pivot(index='eissn', columns='year', values=['Count of Unique Publications', 'Count of Unique Authors'])
pivot_unique_EISSN_publications_df.head(100)

Count of Unique Publications                      \
year                              2019 2020 2021 2022 2023   
eissn                                                        
0148-396X                          1.0  NaN  NaN  NaN  NaN   
0219-1032                          NaN  1.0  NaN  NaN  NaN   
0807-7096                          1.0  NaN  NaN  NaN  NaN   
0893-7400                          1.0  1.0  NaN  1.0  1.0   
0925-4005                          1.0  NaN  NaN  NaN  NaN   
...                                ...  ...  ...  ...  ...   
1097-0177                          NaN  NaN  2.0  NaN  NaN   
1097-0193                          5.0  6.0  7.0  3.0  1.0   
1097-0215                          5.0  7.0  1.0  4.0  2.0   
1097-0223                          1.0  4.0  2.0  2.0  2.0   
1097-0258                          3.0  3.0  7.0  2.0  6.0   

          Count of Unique Authors                       
year                         2019  2020 2021 2022 2023  
eissn                                                   
0148-396X                     1.0   NaN  NaN  NaN  NaN  
0219-1032                     NaN   1.0  NaN  NaN  NaN  
0807-7096                     1.0   NaN  NaN  NaN  NaN  
0893-7400                     1.0   1.0  NaN  1.0  2.0  
0925-4005                     1.0   NaN  NaN  NaN  NaN  
...                           ...   ...  ...  ...  ...  
1097-0177                     NaN   NaN  2.0  NaN  NaN  
1097-0193                    10.0   8.0  8.0  5.0  3.0  
1097-0215                     8.0  13.0  1.0  7.0  2.0  
1097-0223                     1.0   7.0  3.0  6.0  6.0  
1097-0258                     2.0   3.0  6.0  2.0  8.0  

[100 rows x 10 columns]

In [16]:
## Flatten the column names 
pivot_unique_EISSN_publications_df.columns = [f"{x}_{y}" for x, y in pivot_unique_EISSN_publications_df.columns.to_flat_index()]
pivot_unique_EISSN_publications_df.head(100)

,Count of Unique Publications_2019,Count of Unique Publications_2020,Count of Unique Publications_2021,Count of Unique Publications_2022,Count of Unique Publications_2023,Count of Unique Authors_2019,Count of Unique Authors_2020,Count of Unique Authors_2021,Count of Unique Authors_2022,Count of Unique Authors_2023
eissn,,,,,,,,,,
0148-396X,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
0219-1032,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
0807-7096,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
0893-7400,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,2.0
0925-4005,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1097-0177,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN
1097-0193,5.0,6.0,7.0,3.0,1.0,10.0,8.0,8.0,5.0,3.0
1097-0215,5.0,7.0,1.0,4.0,2.0,8.0,13.0,1.0,7.0,2.0


In [17]:
## Identify the column names
# iterating the columns
for col in pivot_unique_EISSN_publications_df.columns:
    print(col)

Count of Unique Publications_2019
Count of Unique Publications_2020
Count of Unique Publications_2021
Count of Unique Publications_2022
Count of Unique Publications_2023
Count of Unique Authors_2019
Count of Unique Authors_2020
Count of Unique Authors_2021
Count of Unique Authors_2022
Count of Unique Authors_2023


In [18]:
## Check for publications with no EISSN but have an ISSN

elements_ISSN_df = elements_data_df[elements_data_df['eissn'].isnull()]
elements_ISSN_df.head()

,group_id,user_id,name,position,department,school,employee_id,net_id,is_current_staff,is_academic,...,issn,eissn,types,type,scopus,web_of_science,crossref,pubmed,europe_pubmed_central,year
14,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,2297-055X,NaN,"Article, Journal",Journal article,2-s2.0-85170700328,WOS:001027272400001,10.3389/fcvm.2021.805278,35004916.0,MED:35004916,2021
26,15,25,Todd Parrish,Professor,Radiology;,Feinberg School of Medicine,1000332,tbp688,1,1,...,2045-2322,NaN,"Article, Journal",Journal article,2-s2.0-85067291814,WOS:000469218200019,10.1038/s41598-019-44416-8,31138878.0,MED:31138878,2019
27,15,25,Todd Parrish,Professor,Radiology;,Feinberg School of Medicine,1000332,tbp688,1,1,...,1471-2474,NaN,"Article, Journal",Journal article,2-s2.0-85067557857,WOS:000472486600001,10.1186/s12891-019-2631-z,31221138.0,MED:31221138,2019
34,15,25,Todd Parrish,Professor,Radiology;,Feinberg School of Medicine,1000332,tbp688,1,1,...,1932-6203,NaN,"Article, Journal",Journal article,2-s2.0-85076053890,WOS:000533987600035,10.1371/journal.pone.0225686,31800609.0,MED:31800609,2019
35,15,25,Todd Parrish,Professor,Radiology;,Feinberg School of Medicine,1000332,tbp688,1,1,...,1932-6203,NaN,"Article, Journal",Journal article,2-s2.0-85076243325,WOS:000534009700053,10.1371/journal.pone.0225759,31805160.0,MED:31805160,2019


In [19]:
## Summarize how many faculty have published in a journal by year
## Make a new dataframe with subset of data, including user_id, publication_id, issn, eissn, and year
## Our backbone ID is the eissn, and then also year
## a) If publication ID is listed more than once for each journal in each year, we only want to count it once. 
## b) If user ID is listed more than once for each journal in each year, we only want to count it once.

unique_ISSN_publications_df = elements_ISSN_df.groupby(['year', 'issn']).agg({'publication_id': 'nunique', 
                                                                          'user_id':'nunique' }).rename(columns={'publication_id': 'Count of Unique Publications',
                                                                                                                'user_id': 'Count of Unique Authors'})
unique_ISSN_publications_df.head(100)


Count of Unique Publications  Count of Unique Authors
year issn                                                            
2019 0002-7863                             1                        2
     0002-9378                             1                        1
     0002-9637                             1                        1
     0003-1348                             3                        2
     0003-2999                            34                       24
...                                      ...                      ...
     1064-3745                             3                        3
     1067-151X                             2                        2
     1070-5503                             1                        2
     1072-7515                             1                        1
     1074-7931                             1                        1

[100 rows x 2 columns]

In [20]:
## Reset the index so year and issn are columns
unique_ISSN_publications_df.reset_index(level=['year','issn'], inplace=True)
unique_ISSN_publications_df.head(100)

,year,issn,Count of Unique Publications,Count of Unique Authors
0,2019,0002-7863,1,2
1,2019,0002-9378,1,1
2,2019,0002-9637,1,1
3,2019,0003-1348,3,2
4,2019,0003-2999,34,24
...,...,...,...,...
95,2019,1064-3745,3,3
96,2019,1067-151X,2,2
97,2019,1070-5503,1,2
98,2019,1072-7515,1,1


In [21]:
## Summarize across 5 years how many unique faculty have published in a journal

unique_ISSN_publications_all_df = elements_ISSN_df.groupby(['issn']).agg({'publication_id': 'nunique', 
                                                                          'user_id':'nunique' }).rename(columns={'publication_id': 'Count of Unique Publications 2019-2023',
                                                                                                                'user_id': 'Count of Unique Authors 2019-2023'})
unique_ISSN_publications_all_df.head(100)

,Count of Unique Publications 2019-2023,Count of Unique Authors 2019-2023
issn,,
0001-6462,1,1
0002-7863,1,2
0002-9378,8,4
0002-9564,1,1
0002-9629,1,1
...,...,...
0300-0729,2,1
0300-483X,1,1
0301-0430,4,3


In [22]:
## Pivot so that the data is provided by Year

pivot_unique_ISSN_publications_df = unique_ISSN_publications_df.pivot(index='issn', columns='year', values=['Count of Unique Publications', 'Count of Unique Authors'])
pivot_unique_ISSN_publications_df.head(100)

Count of Unique Publications                      \
year                              2019 2020 2021 2022 2023   
issn                                                         
0001-6462                          NaN  NaN  1.0  NaN  NaN   
0002-7863                          1.0  NaN  NaN  NaN  NaN   
0002-9378                          1.0  1.0  1.0  4.0  1.0   
0002-9564                          NaN  1.0  NaN  NaN  NaN   
0002-9629                          NaN  NaN  NaN  1.0  NaN   
...                                ...  ...  ...  ...  ...   
0300-0729                          NaN  1.0  NaN  NaN  1.0   
0300-483X                          NaN  1.0  NaN  NaN  NaN   
0301-0430                          2.0  1.0  1.0  NaN  NaN   
0304-3835                          1.0  NaN  NaN  NaN  NaN   
0304-8853                          NaN  NaN  1.0  NaN  NaN   

          Count of Unique Authors                      
year                         2019 2020 2021 2022 2023  
issn                                                   
0001-6462                     NaN  NaN  1.0  NaN  NaN  
0002-7863                     2.0  NaN  NaN  NaN  NaN  
0002-9378                     1.0  1.0  1.0  3.0  1.0  
0002-9564                     NaN  1.0  NaN  NaN  NaN  
0002-9629                     NaN  NaN  NaN  1.0  NaN  
...                           ...  ...  ...  ...  ...  
0300-0729                     NaN  1.0  NaN  NaN  1.0  
0300-483X                     NaN  1.0  NaN  NaN  NaN  
0301-0430                     1.0  1.0  1.0  NaN  NaN  
0304-3835                     1.0  NaN  NaN  NaN  NaN  
0304-8853                     NaN  NaN  1.0  NaN  NaN  

[100 rows x 10 columns]

In [23]:
## Flatten the column names 
pivot_unique_ISSN_publications_df.columns = [f"{x}_{y}" for x, y in pivot_unique_ISSN_publications_df.columns.to_flat_index()]
pivot_unique_ISSN_publications_df.head(100)

,Count of Unique Publications_2019,Count of Unique Publications_2020,Count of Unique Publications_2021,Count of Unique Publications_2022,Count of Unique Publications_2023,Count of Unique Authors_2019,Count of Unique Authors_2020,Count of Unique Authors_2021,Count of Unique Authors_2022,Count of Unique Authors_2023
issn,,,,,,,,,,
0001-6462,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
0002-7863,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
0002-9378,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0
0002-9564,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
0002-9629,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
0300-0729,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
0300-483X,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
0301-0430,2.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN


In [26]:
## Upload the .csv from CMMS of Top ILL & Turnaways

cmms_data_path = "data/2024_05-16_FY24 New Acquisition Candidates Elements Data.csv"
##cmms_data_path = "data/2023_Top ILL & Turnaways_1.csv"

## Read the CSV file and store into Pandas DataFrame 
cmms_data_df = pd.read_csv(cmms_data_path, encoding = "ISO-8859-1")
## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
cmms_data_df.columns =  cmms_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
cmms_data_df.head()

C:\Users\keg827\AppData\Local\Temp\ipykernel_10268\356631709.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cmms_data_df.columns =  cmms_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
C:\Users\keg827\AppData\Local\Temp\ipykernel_10268\356631709.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cmms_data_df.columns =  cmms_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")


,title,nlm_title_abbreviation,print_issn,electronic_issn
0,Rhinology,Rhinology,0300-0729,1996-8604
1,Orthopedics,Orthopedics,0147-7447,1938-2367
2,Journal of asthma,J Asthma,0277-0903,1532-4303
3,Current pharmaceutical design,Curr Pharm Des,1381-6128,1873-4286
4,European journal of dermatology : EJD,Eur J Dermatol,1167-1122,1952-4013


In [27]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(cmms_data_df.index))

Number of rows in the dataframe:  21


In [28]:
## Merge with eissn data
merged_EISSN_df = pd.merge(cmms_data_df, pivot_unique_EISSN_publications_df, left_on='electronic_issn', right_index=True, how="left")
merged_EISSN_df.head()

,title,nlm_title_abbreviation,print_issn,electronic_issn,Count of Unique Publications_2019,Count of Unique Publications_2020,Count of Unique Publications_2021,Count of Unique Publications_2022,Count of Unique Publications_2023,Count of Unique Authors_2019,Count of Unique Authors_2020,Count of Unique Authors_2021,Count of Unique Authors_2022,Count of Unique Authors_2023
0,Rhinology,Rhinology,0300-0729,1996-8604,NaN,1.0,NaN,1.0,2.0,NaN,1.0,NaN,1.0,2.0
1,Orthopedics,Orthopedics,0147-7447,1938-2367,4.0,NaN,3.0,2.0,1.0,7.0,NaN,6.0,2.0,1.0
2,Journal of asthma,J Asthma,0277-0903,1532-4303,NaN,4.0,2.0,NaN,2.0,NaN,7.0,2.0,NaN,2.0
3,Current pharmaceutical design,Curr Pharm Des,1381-6128,1873-4286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,European journal of dermatology : EJD,Eur J Dermatol,1167-1122,1952-4013,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN


In [29]:
## Merge with issn data
merged_ISSN_df = pd.merge(merged_EISSN_df, pivot_unique_ISSN_publications_df, left_on='electronic_issn', right_index=True, how="left")
merged_ISSN_df.head()

,title,nlm_title_abbreviation,print_issn,electronic_issn,Count of Unique Publications_2019_x,Count of Unique Publications_2020_x,Count of Unique Publications_2021_x,Count of Unique Publications_2022_x,Count of Unique Publications_2023_x,Count of Unique Authors_2019_x,...,Count of Unique Publications_2019_y,Count of Unique Publications_2020_y,Count of Unique Publications_2021_y,Count of Unique Publications_2022_y,Count of Unique Publications_2023_y,Count of Unique Authors_2019_y,Count of Unique Authors_2020_y,Count of Unique Authors_2021_y,Count of Unique Authors_2022_y,Count of Unique Authors_2023_y
0,Rhinology,Rhinology,0300-0729,1996-8604,NaN,1.0,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Orthopedics,Orthopedics,0147-7447,1938-2367,4.0,NaN,3.0,2.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Journal of asthma,J Asthma,0277-0903,1532-4303,NaN,4.0,2.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Current pharmaceutical design,Curr Pharm Des,1381-6128,1873-4286,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,European journal of dermatology : EJD,Eur J Dermatol,1167-1122,1952-4013,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(merged_ISSN_df.index))

Number of rows in the dataframe:  21


In [31]:
## Merge with eissn yearly totals

merged_EISSN_total_df = pd.merge(merged_ISSN_df, unique_EISSN_publications_all_df, left_on='electronic_issn', right_index=True, how="left")
merged_EISSN_total_df.head()

,title,nlm_title_abbreviation,print_issn,electronic_issn,Count of Unique Publications_2019_x,Count of Unique Publications_2020_x,Count of Unique Publications_2021_x,Count of Unique Publications_2022_x,Count of Unique Publications_2023_x,Count of Unique Authors_2019_x,...,Count of Unique Publications_2021_y,Count of Unique Publications_2022_y,Count of Unique Publications_2023_y,Count of Unique Authors_2019_y,Count of Unique Authors_2020_y,Count of Unique Authors_2021_y,Count of Unique Authors_2022_y,Count of Unique Authors_2023_y,Count of Unique Publications 2019-2023,Count of Unique Authors 2019-2023
0,Rhinology,Rhinology,0300-0729,1996-8604,NaN,1.0,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0
1,Orthopedics,Orthopedics,0147-7447,1938-2367,4.0,NaN,3.0,2.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,12.0
2,Journal of asthma,J Asthma,0277-0903,1532-4303,NaN,4.0,2.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,9.0
3,Current pharmaceutical design,Curr Pharm Des,1381-6128,1873-4286,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,European journal of dermatology : EJD,Eur J Dermatol,1167-1122,1952-4013,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0


In [32]:
## Merge with issn yearly totals

merged_ISSN_total_df = pd.merge(merged_EISSN_total_df, unique_ISSN_publications_all_df, left_on='electronic_issn', right_index=True, how="left")
merged_ISSN_total_df.head()

,title,nlm_title_abbreviation,print_issn,electronic_issn,Count of Unique Publications_2019_x,Count of Unique Publications_2020_x,Count of Unique Publications_2021_x,Count of Unique Publications_2022_x,Count of Unique Publications_2023_x,Count of Unique Authors_2019_x,...,Count of Unique Publications_2023_y,Count of Unique Authors_2019_y,Count of Unique Authors_2020_y,Count of Unique Authors_2021_y,Count of Unique Authors_2022_y,Count of Unique Authors_2023_y,Count of Unique Publications 2019-2023_x,Count of Unique Authors 2019-2023_x,Count of Unique Publications 2019-2023_y,Count of Unique Authors 2019-2023_y
0,Rhinology,Rhinology,0300-0729,1996-8604,NaN,1.0,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN
1,Orthopedics,Orthopedics,0147-7447,1938-2367,4.0,NaN,3.0,2.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,12.0,NaN,NaN
2,Journal of asthma,J Asthma,0277-0903,1532-4303,NaN,4.0,2.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,9.0,NaN,NaN
3,Current pharmaceutical design,Curr Pharm Des,1381-6128,1873-4286,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,European journal of dermatology : EJD,Eur J Dermatol,1167-1122,1952-4013,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN


In [33]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(merged_ISSN_total_df.index))

Number of rows in the dataframe:  21


In [34]:
## Save dataframe to a CSV

with open(r"output/merged_ISSN_total_df.csv", 'w', encoding='utf-8') as file:
    merged_ISSN_total_df.to_csv(file, line_terminator='\n', index=True)
    file.close()